LLM application through query routing and long-context, a type of reflection flow based on the user query
===

RAG routing is based on the complexity of the query and the context. The routing follows these rules:

1. Simple query, direct answer, no routing.
2. Answerable question, but complex query, then routing for query decomposition.
3. Route to long-context query for unanswerable questions.

Origin paper: [Retrieval Augmented Generation or Long-Context LLMs? A Comprehensive Study and Hybrid Approach](https://arxiv.org/abs/2407.16833)

Impl. by: LLama-Index


In [1]:
from rich.pretty import pprint as pp
from icecream import ic
from IPython.display import Markdown

import nest_asyncio
nest_asyncio.apply()

## Const

In [2]:
from llama_index.core.response_synthesizers import ResponseMode

llm = "llama3-70b-8192" # Gemini for "models/gemini-1.5-pro" # Grop for "llama3-70b-8192", "llama-3.1-70b-versatile", "llama-3.1-8b-instant"
embeds = "models/text-embedding-004"
chunk_size = 512
MAX_TOKENS = 2048
data_urls = [
    "https://hermesworld.com/de/karriere/jobs/Junior-Manager-mwd-HR-Controlling-de-j4364.html",
    "https://hermesworld.com/de/karriere/jobs/Senior-Product-Manager-mwd-de-j3839.html",
]
rerank_top_k = 10
similarity_top_k = 10
num_multi_steps = 4
verbose = True
streaming = False
response_mode = ResponseMode.TREE_SUMMARIZE
# https://github.com/run-llama/llama_index/blob/71c2cfdfbec6bcdd71f0e39f5dbb52c0e9f68ae5/llama-index-core/llama_index/core/response_synthesizers/type.py

## Token counting

In [3]:
from llama_index.core.callbacks import TokenCountingHandler
import tiktoken

token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode
)

In [4]:
def print_token_counter(counter: TokenCountingHandler):
    pp(
        (
            "Embedding Tokens: ",
            counter.total_embedding_token_count,
            "LLM Prompt Tokens: ",
            counter.prompt_llm_token_count,
            "LLM Completion Tokens: ",
            counter.completion_llm_token_count,
            "Total LLM Token Count: ",
            counter.total_llm_token_count,
        )
    )

    counter.reset_counts()


print_token_counter(token_counter)

('Embedding Tokens: ', 0, 'LLM Prompt Tokens: ', 0, 'LLM Completion Tokens: ', 0, 'Total LLM Token Count: ', 0)

## LLama-Index setting

In [5]:
from llama_index.core import Settings
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini
from llama_index.core.callbacks import CallbackManager
from llama_index.llms.groq import Groq

Settings.llm = Groq(
    model=llm, temperature=0, max_tokens=MAX_TOKENS
)  # or use Groq model
Settings.embed_model = GeminiEmbedding(model_name=embeds)
Settings.callbacks = CallbackManager([token_counter])

In [6]:
from llama_index.readers.web import SimpleWebPageReader
import tempfile
from llama_parse import LlamaParse

origin_documents = SimpleWebPageReader(html_to_text=True).load_data(data_urls)
# pp(origin_documents)
document_contents = "\n".join(
    [f"----document 1:----\n\n{doc.text}" for doc in origin_documents]
)
# print(document_contents)
temp_file = tempfile.NamedTemporaryFile(delete=True, suffix=".txt")
temp_file.write(document_contents.encode("utf-8"))
temp_file.seek(0) 
ic(temp_file.name)
print(temp_file.read()) 
parser = LlamaParse(
    # api_key= api_key,
    result_type="markdown"
) 
documents = parser.load_data(temp_file.name)
pp(documents)
temp_file.close()

ic| temp_file.name: '/tmp/tmpywhmtr0r.txt'


b'----document 1:----\n\n[ Privatkunden ](https://www.myhermes.de)\n\n\xc3\x96sterreich China Deutschland Vereinigtes K\xc3\xb6nigreich International USA\n\nMenu AT CN DE EN INT US [Hermes](https://www.hermesworld.com/de/)\n\n  * navigation.search\n\n  * [Start](https://www.hermesworld.com/de/)\n  * [Unsere Dienstleistungen](https://www.hermesworld.com/de/unsere-dienstleistungen/)\n\n    * [Transport Logistics](https://www.hermesworld.com/de/unsere-dienstleistungen/transport-logistics/)\n      * [Supply Chain Solutions](https://www.hermesworld.com/de/unsere-dienstleistungen/transport-logistics/supply-chain-solutions/unsere-fokusbranchen/)\n        * [Unsere Fokusbranchen](https://www.hermesworld.com/de/unsere-dienstleistungen/transport-logistics/supply-chain-solutions/unsere-fokusbranchen/)\n        * [Supply Chain Services](https://www.hermesworld.com/de/unsere-dienstleistungen/transport-logistics/supply-chain-solutions/supply-chain-services/)\n        * [E-Services](https://www.herme

[
│   Document(
│   │   id_='b3f2fa21-824a-4ec3-87bb-e109447ec6f4',
│   │   embedding=None,
│   │   metadata={},
│   │   excluded_embed_metadata_keys=[],
│   │   excluded_llm_metadata_keys=[],
│   │   relationships={},
│   │   text='# Privatkunden\n\nÖsterreich China Deutschland Vereinigtes Königreich International USA\n\nMenu AT CN DE EN INT US Hermes\n\n- navigation.search\n- Start\n- Unsere Dienstleistungen',
│   │   mimetype='text/plain',
│   │   start_char_idx=None,
│   │   end_char_idx=None,
│   │   text_template='{metadata_str}\n\n{content}',
│   │   metadata_template='{key}: {value}',
│   │   metadata_seperator='\n'
│   ),
│   Document(
│   │   id_='70d527f4-ebed-4e97-ab2f-568e56a23a07',
│   │   embedding=None,
│   │   metadata={},
│   │   excluded_embed_metadata_keys=[],
│   │   excluded_llm_metadata_keys=[],
│   │   relationships={},
│   │   text='# Zusammenarbeit\n\nhttps://www.hermesworld.com/de/unsere-dienstleistungen/distribution/2-mann-handling/zusammenarbeit/\n\n# Dropshipping\n\nhttps://www.hermesworld.com/de/unsere-dienstleistungen/distribution/2-mann-handling/dropshipping/\n\n# Unser Service für Versender\n\nhttps://www.hermesworld.com/de/unsere-dienstleistungen/distribution/2-mann-handling/unser-service-fuer-versender/\n\n# Unser Service für Endkunden\n\nhttps://www.hermesworld.com/de/unsere-dienstleistungen/distribution/2-mann-handling/unser-service-fuer-endkunden/\n\n# CO2-neutraler Versand\n\nhttps://www.hermesworld.com/de/unsere-dienstleistungen/distribution/2-mann-handling/klimaneutraler-versand/\n\n# HES Pay\n\nhttps://www.hermesworld.com/de/unsere-dienstleistungen/distribution/2-mann-handling/hes-pay/\n\n# Distribution in Europa\n\nhttps://www.hermesworld.com/de/unsere-dienstleistungen/distribution/distribution-in-europa/\n\n# Über uns\n\n# Auf einen Blick\n\nhttps://www.hermesworld.com/de/ueber-uns/auf-einen-blick/hermes-auf-einen-blick/\n\n# Hermes Gruppe\n\n# Hermes Einrichtungs Service\n\nhttps://www.hermesworld.com/de/ueber-uns/hermes-gruppe/hermes-einrichtungs-service/\n\n- Hermes Einrichtungs Service\n- Logistikzentren\n- Auslieferdepots\n\n# Hermes Fulfilment\n\nhttps://www.hermesworld.com/de/ueber-uns/hermes-gruppe/hermes-fulfilment/hermes-fulfilment/\n\n- Hermes Fulfilment\n- Daten & Fakten\n- Einblicke\n- Nachhaltigkeit\n\n# Hermes Logistik Österreich\n\nhttps://www.hermesworld.com/de/ueber-uns/hermes-gruppe/hermes-logistik-at/hermes-logistik-oesterreich/\n\n- Hermes Logistik Österreich\n- PaketShop Partner\n\n# Hermes Germany GmbH\n\nhttps://www.hermesworld.com/de/ueber-uns/hermes-gruppe/hermes-logistik-gruppe-deutschland/hermes-logistik-gruppe-deutschland/\n\n- Hermes Germany GmbH\n- PaketShop-Partner werden\n- Vertragspartner werden\n\n- Formular\n\n# Hermes NexTec\n\nhttps://www.hermesworld.com/de/ueber-uns/hermes-gruppe/hermes-nextec/\n\n# Hermes Germany\n\nhttps://www.hermesworld.com/de/ueber-uns/hermes-',
│   │   mimetype='text/plain',
│   │   start_char_idx=None,
│   │   end_char_idx=None,
│   │   text_template='{metadata_str}\n\n{content}',
│   │   metadata_template='{key}: {value}',
│   │   metadata_seperator='\n'
│   ),
│   Document(
│   │   id_='f1777754-ee80-4119-896b-6231c872d73c',
│   │   embedding=None,
│   │   metadata={},
│   │   excluded_embed_metadata_keys=[],
│   │   excluded_llm_metadata_keys=[],
│   │   relationships={},
│   │   text='# gruppe/hermes-transport-logistics/hermes-transport-logistics/\n\n- Hermes Germany\n- Downloads\n\n# Hermes UK\n\n- Hermes UK\n- Kundenstimmen\n\n# Hermes BorderGuru\n\n- Hermes BorderGuru\n\n# Management\n\n- Management\n\n# Klima- und Umweltschutz\n\n- Klima- und Umweltschutz\n- Grüne Zustellung\n- Energieeffiziente Standorte\n- Unterstützung von Klimaschutzprojekten\n\n# Soziale Verantwortung\n\n- Faire Zusammenarbeit\n- Einhaltung von Sozialstandards\n- Gesellschaftliches Engagement\n\n# Downloads\n\n- Plattformen & Bezahlfunktionen China\n- Export nach China\n\n# Messen\n\n- Messen\n\n# Historie\n\n- 2010er Jahre\n- 2000er Jahre\n

## Setup step methods

### Chain

In [7]:
from llama_index.core import get_response_synthesizer

synthesizer = get_response_synthesizer(
    response_mode=response_mode, streaming=streaming
)

### Prompting

Re-prompt the default prompt for German input data. This can enhance query performance.

In [8]:
from langchain import hub
from llama_index.core import PromptTemplate

prompt_str = hub.pull("hwchase17/llama-rag").template.replace("context", "context_str").replace("question", "query_str")
prompt_str = prompt_str.replace("[INST] <<SYS>>", "[INST] <<SYS>> You contains data in German, thinking everything including query and ansewr in German if possible. ")
pp(prompt_str)

prompt_tmpl = PromptTemplate(prompt_str)

'[INST] <<SYS>> You contains data in German, thinking everything including query and ansewr in German if possible. Answer the users query_str only taking into account the following context_str. If the user asks for information not found in the below context_str, do not answer.\n\n<context_str>\n{context_str}\n</context_str>\n<</SYS>>\n\n {query_str} [/INST]'

### Top K RAG

In [9]:
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.postprocessor import SentenceTransformerRerank


splitter = SentenceSplitter(chunk_size=chunk_size)
vector_index = VectorStoreIndex.from_documents(documents, transformations=[splitter])
vector_query_engine = vector_index.as_query_engine(
    similarity_top_k=similarity_top_k,
    response_synthesizer=synthesizer,
    node_postprocessors=[
        SentenceTransformerRerank(top_n=rerank_top_k, model="BAAI/bge-reranker-base")
    ],
    summary_template=prompt_tmpl,
)
vector_query_engine.update_prompts(
    {"response_synthesizer:summary_template": prompt_tmpl}
)

In [10]:
prompts_dict = vector_query_engine.get_prompts()
pp(list(prompts_dict.values()))

[
│   PromptTemplate(
│   │   metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>},
│   │   template_vars=['context_str', 'query_str'],
│   │   kwargs={},
│   │   output_parser=None,
│   │   template_var_mappings=None,
│   │   function_mappings=None,
│   │   template='[INST] <<SYS>> You contains data in German, thinking everything including query and ansewr in German if possible. Answer the users query_str only taking into account the following context_str. If the user asks for information not found in the below context_str, do not answer.\n\n<context_str>\n{context_str}\n</context_str>\n<</SYS>>\n\n {query_str} [/INST]'
│   )
]

### Multi-step query

In [11]:
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)
from llama_index.core.query_engine import MultiStepQueryEngine

step_decompose_transform = StepDecomposeQueryTransform(verbose=verbose)
multi_steps_query_engine = MultiStepQueryEngine(
    query_engine=vector_query_engine,
    query_transform=step_decompose_transform,
    response_synthesizer=synthesizer,
    num_steps=num_multi_steps,
)

### Long context query

In [12]:
from llama_index.core.llms.llm import LLM
from llama_index.core.query_engine import CustomQueryEngine
from typing import Any
from langchain import hub
from llama_index.core import PromptTemplate


class VanillaQueryEngine(CustomQueryEngine):
    """RAG String Query Engine."""

    llm: LLM
    context: str
    prompt_tmpl: PromptTemplate

    def __call__(self, *args: Any, **kwds: Any) -> str:
        return self.custom_query(*args, **kwds)

    def custom_query(self, query_str: str) -> str:
        full_query = self.prompt_tmpl.format(
            context_str=self.context, query_str=query_str
        )
        return str(self.llm.complete(full_query))


lc_query_engine = VanillaQueryEngine(
    response_synthesizer=synthesizer,
    llm=Settings.llm,
    context="\n".join([doc.text for doc in documents]),
    prompt_tmpl=prompt_tmpl,
)

## Query with methods

### Simple query for try

In [13]:
query = "Tell me the jobs you know at Hermes Group?"

### Top K RAG

In [14]:
vc_res = vector_query_engine.query(query)
print(vc_res.response)

Hallo!

Laut dem Kontext gibt es bei der Hermes Gruppe folgende Jobs:

1. (Junior) Manager (m/w/d) HR Controlling
2. Logistik & gewerbliche Jobs
3. Jobs beim HES (Hermes Einrichtungs Service)
4. Jobs beim HES in Ansbach
5. Jobs in unseren Depots


### Multi-step query

In [15]:
ms_res = multi_steps_query_engine.query(query)
print(ms_res.response)

> Current query: Tell me the jobs you know at Hermes Group?
> New query: Since there is no knowledge source context provided, the answer is:

None
Empty Response


In [16]:
def show_multi_steps(ms_res):
    sub_qa = ms_res.metadata["sub_qa"]
    tuples = [(t[0], t[1].response) for t in sub_qa]
    pp(tuples)

show_multi_steps(ms_res)

[]

### Long context query

In [17]:
lc_res = lc_query_engine(query)
ic(lc_res)

ic| lc_res: ('Based on the provided context, I can identify the following job openings at '
             'Hermes Group:
            '
             '
            '
             '1. (Junior) Manager (m/w/d) HR Controlling
            '
             '2. (Senior-) Product Manager (m/w/d)
            '
             '
            '
             'Additionally, there are mentions of various departments and teams within '
             'Hermes Group, such as:
            '
             '
            '
             '* Hermes Fulfilment
            '
             '* Hermes Germany
            '
             '* Hermes Einrichtungs Service
            '
             '* Hermes Logistik Österreich
            '
             '* Hermes UK
            '
             '* Hermes BorderGuru
            '
             '* Management
            '
             '* Klima- und Umweltschutz
            '
             '* Soziale Verantwortung
            '
             '* Downloads
            '
             '* Mess

'Based on the provided context, I can identify the following job openings at Hermes Group:\n\n1. (Junior) Manager (m/w/d) HR Controlling\n2. (Senior-) Product Manager (m/w/d)\n\nAdditionally, there are mentions of various departments and teams within Hermes Group, such as:\n\n* Hermes Fulfilment\n* Hermes Germany\n* Hermes Einrichtungs Service\n* Hermes Logistik Österreich\n* Hermes UK\n* Hermes BorderGuru\n* Management\n* Klima- und Umweltschutz\n* Soziale Verantwortung\n* Downloads\n* Messen\n* Historie\n* Karriere\n* Hermes Europe\n* Architecture & Information Security\n* ANS\n* Cloud & Application Infrastructure\n* IT-Finance und IT-Licensemanagement\n* BlueOps\n* European Network\n* Black OPs\n* Business Analytics\n* Support Solutions\n* Servicedesk\n* Supply Chain Solutions\n* Logistik & gewerbliche Jobs\n\nPlease note that these might not be specific job openings, but rather departments or teams within the company.'

# Replication query router

The routing is ochstrated by the complexity of the query and the context. Because of the complexity of the query, the RAG will be failed for some reason. The routing will be used to decompose the query into multiple steps. The long-context query will be used for unanswerable questions.

## RAG can be failed by 4 reasons

Accoriding to the paper, there are four reasons why RAG can be failed.

The four reasons include: 

(A) The query requires multi-step reasoning so the results of previous steps are needed to retrieve information for later steps, e.g. ‘‘What nationality is the performer of song XXX’’. 

(B) The query is general, e.g. ‘‘What does the group think about XXX’’, which is challenging for the retriever to formulate a good query. 

(C) The query is long and complex, which is challenging for the retriever to understand. However, answering this kind of questions is arguably, an advantage of LLMs. 

(D) The query is implicit, demanding a thorough understanding of the entire context. For instance, in a lengthy conversational narrative about a space voyage, a question like ‘‘What caused the shadow behind the spaceship?’’ requires readers to connect the dots and deduce the answer, as there is no explicit mention of the shadow when the cause is revealed.

##### Short to say:

(A) The query requires multi-step reasoning.

(B) The query is general. 

(C) The query is long and complex, requiring LLMs. 

(D) The query is implicit, demanding a thorough understanding of the entire context.



## Setup route query engine

### Tools

In [18]:
from llama_index.core.tools import QueryEngineTool

vector_query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for simple and answerable questions.",
    return_direct=True,
)

multi_steps_query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=multi_steps_query_engine,
    description="Useful for answerable questions that require multiple step reasonings.",
    return_direct=True,
)

lc_query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=lc_query_engine,
    description="Useful for long and complex or implicit questions or queries that require full context to obtain results.",
    return_direct=True,
)

query_engine_tools = [
    vector_query_engine_tool,
    multi_steps_query_engine_tool,
    lc_query_engine_tool,
]

### Router query engine

#### Router selection prompting

The routing choice is determined by the query's complexity and context, which can trigger errors for various reasons. 

This serves as the prompt for conducting error analysis on the routing choice, we can modify the origin content and migrate to the Llama-Index [LLMSingleSelector](https://github.com/run-llama/llama_index/blob/71c2cfdfbec6bcdd71f0e39f5dbb52c0e9f68ae5/llama-index-core/llama_index/core/selectors/llm_selectors.py).

Origin prompt for the error analysis in paper:

```text

You are given some text chunks from an article, and a question. The text chunks are retrieved by an external retriever.
Now:
(1) Tell whether the question can be answered based only on the provided text chunks.
(2) If the question can be answered, answer the question based on the texts as concisely as you can, using a single
phrase if possible.
(3) If the question cannot be answered, choose the reason from the following:
A. The question needs multistep reasoning, thus it is hard to retrieve all the relevant chunks. For example, "What
nationality is the performer of song You Can?" contains two steps: find the performer, then find the nationality of the
performer. Other examples include "Where does the director of film Wine Of Morning work at?", "What is another
notable work made by the author of Miss Sara Sampson?"
B. The question is a general query, thus it is hard to retrieve relevant chunks. For example, "What did the group
think about Dave leaving?" is general because the group may include multiple persons, and they can have different
thinkings.
C. The question is long and complex, which is hard for the retriever to encode it to retrieve relevant chunks. For
example, "What did Julie Morgan elaborate on the online survey when talking about the evaluations on the legitimacy
of the children’s rights, protection and demands?", "The Huskies football team were invited to the Alamo Bowl where
they were defeated by a team coached by Art Briles and who played their home games at what stadium?"
D. The question is not explicit and requires comprehensive understanding of the whole story and cannot be solved
using retrieval-augmented generation. For example, "What caused the shadow behind Koerber’s ship?" needs a
comprehensive understanding of the whole story. Another example like "How many words are there in the article"
also requires the complete article.
E. Others.
Keep the above reasons in mind, and choose the most possible reason if you think the question cannot be answered
based on the text. Output the results in JSON format.
{in_context_examples}
Text: {context}
Question: {input}
Answer:

```

We can modifiy this into:

In [19]:
modified_error_anlysis = """You receive text chunks from various data sources along with a question or query. These chunks are obtained through an external retriever or query engine.
Now:
(1) Think about whether the question or the query can be simply answered based only on the provided text chunks.
(2) If the question or the query can be answered, answer the question or the query based on the texts as concisely as you can, using a single
phrase if possible.
(3) If the question or the query cannot be answered, choose the reason from the following:
A. The question or the query needs multi-step reasoning, thus it is hard to retrieve all the relevant chunks. For example, "What
nationality is the performer of song You Can?" contains two steps: find the performer, then find the nationality of the
performer. Other examples include "Where does the director of film Wine Of Morning work at?", "What is another
notable work made by the author of Miss Sara Sampson?"
B. The question or the query is a general query, thus it is hard to retrieve relevant chunks. For example, "What did the group
think about Dave leaving?" is general because the group may include multiple persons, and they can have different
thinkings.
C. The question or the query is long and complex, which is hard for the retriever to encode it to retrieve relevant chunks. For
example, "What did Julie Morgan elaborate on the online survey when talking about the evaluations on the legitimacy
of the children’s rights, protection and demands?", "The Huskies football team were invited to the Alamo Bowl where
they were defeated by a team coached by Art Briles and who played their home games at what stadium?"
D. The question or the query is not explicit and requires comprehensive understanding of the whole story and cannot be solved
using retrieval-augmented generation. For example, "What caused the shadow behind Koerber’s ship?" needs a
comprehensive understanding of the whole story. Another example like "How many words are there in the article"
also requires the complete article.
E. Others.

Keep the above reasons in mind, and choose the most possible reason(s) with following rule to handle the question or the query that cannot be answered
based on the text:

For error A and B and E, choose the multi-step query engine.
For error C and D, choose the query engine that can handle long and complex or implicit questions or queries.
Otherwise, just consider that the question or the query is simply answerable based on the provided text chunks.

Notice that text chunks are in German and the question or the query might be in English, please translate the it into German and think in German style if necessary.
Here is the instruction with choices for the selection task:
"""

In [20]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors.llm_selectors import LLMSingleSelector
from llama_index.core.response_synthesizers import TreeSummarize

SINGLE_SELECT_PROMPT_TMPL = modified_error_anlysis + (
    "Some choices are given below. It is provided in a numbered list "
    "(1 to {num_choices}), "
    "where each item in the list corresponds to a summary.\n"
    "---------------------\n"
    "{context_list}"
    "\n---------------------\n"
    "Using only the choices above and not prior knowledge, return "
    "the choice that is most relevant to the question: '{query_str}'\n"
)

print(SINGLE_SELECT_PROMPT_TMPL)

router_query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(
        prompt_template_str=SINGLE_SELECT_PROMPT_TMPL
    ),
    query_engine_tools=query_engine_tools,
    summarizer=TreeSummarize(
        streaming=streaming,
        use_async=True,
        verbose=verbose,
    ),
    verbose=verbose,
)

You receive text chunks from various data sources along with a question or query. These chunks are obtained through an external retriever or query engine.
Now:
(1) Think about whether the question or the query can be simply answered based only on the provided text chunks.
(2) If the question or the query can be answered, answer the question or the query based on the texts as concisely as you can, using a single
phrase if possible.
(3) If the question or the query cannot be answered, choose the reason from the following:
A. The question or the query needs multi-step reasoning, thus it is hard to retrieve all the relevant chunks. For example, "What
nationality is the performer of song You Can?" contains two steps: find the performer, then find the nationality of the
performer. Other examples include "Where does the director of film Wine Of Morning work at?", "What is another
notable work made by the author of Miss Sara Sampson?"
B. The question or the query is a general query, thus it is

#### Simple question

In [21]:
simply_query = "What data do you have?"
route_res = router_query_engine.query(simply_query)
Markdown(route_res.response)

Selecting query engine 0: The question 'What data do you have?' is a simple and direct question that can be answered based on the provided text chunks..


Hallo! Ich habe Daten über die Hermes Gruppe, ein international tätiges Logistikunternehmen. Insbesondere habe ich Informationen über den Service für Endkunden, CO2-neutralen Versand, HES Pay, Distribution in Europa, Hermes Gruppe, Hermes Einrichtungs Service, Hermes Fulfilment, Hermes Logistik Österreich und Hermes Germany GmbH.

In [22]:
lc_res = lc_query_engine.query(simply_query)
Markdown(lc_res.response)

Hallo! Ich habe Daten über die Hermes Gruppe, einem Logistikunternehmen. Meine Daten umfassen Informationen über die Dienstleistungen, Unternehmen, Karriere, Kontakt, Tools und Services, Über uns, Historie, Karriere, Newsroom, und vieles mehr. Ich bin bereit, deine Fragen zu beantworten!

#### Mult-step reasoning

In [32]:
normal_query = "What is (Senior) Product Manager or (Junior) Manager HR Controlling, return final result in a markdown table (two rows for each job and two columns for job title, task) respectively."  
route_res = router_query_engine.query(normal_query)
Markdown(route_res.response)

Selecting query engine 1: The question requires multi-step reasoning to retrieve the relevant chunks, as it involves identifying the tasks for two different job titles, (Senior) Product Manager and (Junior) Manager HR Controlling..
> Current query: What is (Senior) Product Manager or (Junior) Manager HR Controlling, return final result in a markdown table (two rows for each job and two columns for job title, task) respectively.
> New query: Since there is no knowledge source context provided, the next question would be:

What are the typical responsibilities and tasks of a (Senior) Product Manager and a (Junior) Manager HR Controlling?
> Current query: What is (Senior) Product Manager or (Junior) Manager HR Controlling, return final result in a markdown table (two rows for each job and two columns for job title, task) respectively.
> New query: Based on the provided context and previous reasoning, the next question that can be answered from the context is:

Can the typical responsibili

In [24]:
lc_res = lc_query_engine.query(normal_query)
Markdown(lc_res.response)

Here is the information about the (Junior) Manager (m/w/d) and HR Controlling position:

**Job Title:** (Junior) Manager (m/w/d) HR Controlling

**Job Description:**

* Create evaluations, statistics, and presentations for internal stakeholders such as HR colleagues or departments, as well as for external stakeholders like authorities
* Accompany processes (e.g. salary round) and HR projects to enable data-based decisions
* Work actively on building a quality control system between our systems
* Bring in new ideas and improvement suggestions with conviction

**Requirements:**

* Completed studies in economics, social sciences, or human resources management
* First professional experience in the form of internships, student jobs, or part-time jobs
* Fun working with data and large amounts of numbers
* Interest in modern personnel work
* Problem-solving skills, self-organization, and goal orientation are among your strengths
* Secure application of common MS Office programs (especially Excel and PowerPoint) is an advantage

**What we offer:**

* Use of leading technologies and agile processes
* You work for a multiple award-winning employer whose corporate culture is characterized by an open and cooperative togetherness
* Flexible working hours, with the possibility of mobile working after arrangement
* Comprehensive training offer
* 15% discount on otto.de and other shops
* Subsidy of the Deutschlandticket, old-age provision, JobRad, various sports and leisure offers, etc.
* Discounted and varied meal options in our modern canteen

Here is the information in a markdown table:

| **Job Title** | **Job Description** | **Requirements** | **What we offer** |
| --- | --- | --- | --- |
| (Junior) Manager (m/w/d) HR Controlling | Create evaluations, statistics, and presentations for internal and external stakeholders; Accompany processes and HR projects; Work on building a quality control system | Completed studies in economics, social sciences, or human resources management; First professional experience; Fun working with data; Interest in modern personnel work; Problem-solving skills, self-organization, and goal orientation | Use of leading technologies and agile processes; Flexible working hours; Comprehensive training offer; 15% discount on otto.de and other shops; Subsidy of the Deutschlandticket, old-age provision, JobRad, etc. |

#### Long and complex question

In [26]:
long_complex_query="Tell me the information of the jobs within your knowledge base, including TODO and the benefits for working in the Hermes Group, response in a markdown table."
route_res = router_query_engine.query(long_complex_query)
Markdown(route_res.response)

Selecting query engine 2: The question or the query is long and complex, which is hard for the retriever to encode it to retrieve relevant chunks..


Here is the information about the jobs within the knowledge base, including the TODO and the benefits for working in the Hermes Group, in a markdown table:

| **Job Title** | **Company** | **Location** | **Job Description** | **Requirements** | **Benefits** |
| --- | --- | --- | --- | --- | --- |
| (Junior) Manager (m/w/d) HR Controlling | Hermes Germany GmbH | Hermes Zentrale Hamburg | Create reports, statistics, and presentations for internal stakeholders; support processes and HR projects with data-based decisions | Study in economics, social sciences, or personnel management; first work experience; proficiency in MS Office | Flexible working hours, mobile working possible; comprehensive training offer; 15% discount on otto.de and other shops; company pension scheme, job bike, various sports and leisure activities |
| (Senior-) Product Manager (m/w/d) | Hermes Germany GmbH | Hermes Zentrale Hamburg | Strategically and operationally responsible for a part of the product portfolio; analyze target groups, market, and competition; initiate and implement product, price, and communication measures | Study in product management or similar; several years of experience in product or brand management; analytical skills, customer orientation, creativity, and negotiation skills | Flexible working hours, mobile working possible; comprehensive training offer; 15% discount on otto.de and other shops; company pension scheme, job bike, various sports and leisure activities |

**TODO:**

* There are two job postings mentioned in the context, but only two job titles are extracted. There might be more job postings in the context that are not extracted.
* The job descriptions, requirements, and benefits are not exhaustive, as they are only extracted from the two job postings mentioned.

**Benefits for working in the Hermes Group:**

* Flexible working hours, mobile working possible
* Comprehensive training offer
* 15% discount on otto.de and other shops
* Company pension scheme
* Job bike
* Various sports and leisure activities
* International work environment with a multicultural atmosphere
* Long-term perspectives and daily changing challenges in a secure and future-oriented industry